In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yyt0817","key":"93acccd48fc47c9f13117065c32c5c6d"}'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json  ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d wilmerarltstrmberg/recipe-dataset-over-2m

100% 633M/635M [00:29<00:00, 23.9MB/s]
100% 635M/635M [00:29<00:00, 22.4MB/s]


In [ ]:
!unzip -q recipe-dataset-over-2m.zip

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("recipes_data.csv")

In [ ]:
data.head()

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com


In [ ]:
from numpy.random import default_rng
data = data[0:1000]

In [ ]:
#get rid of useless attributes: site, source, link
data = data.drop(['site', 'source', 'link'], axis=1)

In [ ]:
data.head()

,title,ingredients,directions,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""bite size shredded rice biscuits"", ""vanilla""..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""cream of mushroom soup"", ""beef"", ""sour cream..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""pepper"", ""cream cheese"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken gravy"", ""cream of mushroom soup"", ""c..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""graham cracker crumbs"", ""powdered sugar"", ""p..."


In [ ]:
import ast
# Convert the string representation of the list into an actual list
data['directions'] = data['directions'].apply(ast.literal_eval)
data['NER'] = data['NER'].apply(ast.literal_eval)

In [ ]:
#get all unique ingredients
ingredients_vocab = set(ingredient for sublist in data['NER'] for ingredient in sublist)

In [ ]:
def preprocess(strings_list):
    return [string.replace(", ", " , ").replace(".", " . ") for string in strings_list]

In [ ]:
data['directions'] = data['directions'].apply(preprocess)

In [ ]:
ingredients_list = list(data['NER'])
directions_list = list(data['directions'])
title_list = list(data['title'])

In [ ]:
def preprocess_dataset(ingredients_list, directions_list, title_list):
    # Initialize an empty list to store preprocessed data
    question = []
    answer = []

    # Iterate over each recipe in the dataset
    for ingredients, directions, title in zip(ingredients_list, directions_list, title_list):
        # Combine ingredients and directions into a single string with spaces between words/symbols
        ingredients_text = ' , '.join(ingredients)
        directions_text = ' '.join(directions)
        question.append(f"What dishes can we make with {ingredients_text}")
        answer.append(f"You can make {title} .  Here is the recipe : {directions_text}")

    return question, answer

In [ ]:
question, answer = preprocess_dataset(ingredients_list, directions_list, title_list)

In [ ]:
df = pd.DataFrame(list(zip(question, answer)),
               columns =['question', 'answer'])

In [ ]:
df.head()

,question,answer
0,What dishes can we make with bite size shredde...,You can make No-Bake Nut Cookies . Here is th...
1,What dishes can we make with cream of mushroom...,You can make Jewell Ball'S Chicken . Here is ...
2,"What dishes can we make with frozen corn , pep...",You can make Creamy Corn . Here is the recipe...
3,"What dishes can we make with chicken gravy , c...",You can make Chicken Funny . Here is the reci...
4,What dishes can we make with graham cracker cr...,You can make Reeses Cups(Candy) . Here is t...


In [ ]:
df.to_csv('/content/drive/My Drive/recipe_dataset.csv', index=False)